##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Voice memos

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Voice_memos.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook provides a quick example of how to work with audio and text files in the same prompt. You'll use the Gemini API to help you generate ideas for your next blog post, based on voice memos you recorded on your phone, and previous articles you've written.

In [2]:
%pip install -U -q "google-genai>=1.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 7.9 MB/s eta 0:00:00


### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

Install PDF processing tools.

In [4]:
!apt install -q poppler-utils

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (141 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


## Upload your audio and text files


In [5]:
!wget -q -O Walking_thoughts_3.m4a https://storage.googleapis.com/generativeai-downloads/data/Walking_thoughts_3.m4a
!wget -q -O A_Possible_Future_for_Online_Content.pdf https://storage.googleapis.com/generativeai-downloads/data/A_Possible_Future_for_Online_Content.pdf
!wget -q -O Unanswered_Questions_and_Endless_Possibilities.pdf https://storage.googleapis.com/generativeai-downloads/data/Unanswered_Questions_and_Endless_Possibilities.pdf

In [6]:
audio_file_name = "Walking_thoughts_3.m4a"
audio_file = client.files.upload(file=audio_file_name)

## Extract text from the PDFs

In [7]:
!pdftotext A_Possible_Future_for_Online_Content.pdf
!pdftotext Unanswered_Questions_and_Endless_Possibilities.pdf

In [8]:
blog_file_name = "A_Possible_Future_for_Online_Content.txt"
blog_file = client.files.upload(file=blog_file_name)

In [9]:
blog_file_name2 = "Unanswered_Questions_and_Endless_Possibilities.txt"
blog_file2 = client.files.upload(file=blog_file_name2)

## System instructions

Write a detailed system instruction to configure the model.

In [10]:
si="""Objective: Transform raw thoughts and ideas into polished, engaging blog posts that capture a writers unique style and voice.
Input:
Example Blog Posts (1-5): A user will provide examples of blog posts that resonate with their desired style and tone. These will guide you in understanding the preferences for word choice, sentence structure, and overall voice.
Audio Clips: A user will share a selection of brainstorming thoughts and key points through audio recordings. They will talk freely and openly, as if they were explaining their ideas to a friend.
Output:
Blog Post Draft: A well-structured first draft of the blog post, suitable for platforms like Substack or LinkedIn.
The draft will include:
Clear and engaging writing: you will strive to make the writing clear, concise, and interesting for the target audience.
Tone and style alignment: The language and style will closely match the examples provided, ensuring consistency with the desired voice.
Logical flow and structure: The draft will be organized with clear sections based on the content of the post.
Target word count: Aim for 500-800 words, but this can be adjusted based on user preferences.
Process:
Style Analysis: Carefully analyze the example blog posts provided by the user to identify key elements of their preferred style, including:
Vocabulary and word choice: Formal vs. informal, technical terms, slang, etc.
Sentence structure and length: Short and impactful vs. longer and descriptive sentences.
Tone and voice: Humorous, serious, informative, persuasive, etc.
Audio Transcription and Comprehension: Your audio clips will be transcribed with high accuracy. you will analyze them to extract key ideas, arguments, and supporting points.
Draft Generation: Using the insights from the audio and the style guidelines from the examples, you will generate a first draft of the blog post. This draft will include all relevant sections with supporting arguments or evidence, and a great ending that ties everything together and makes the reader want to invest in future readings.
"""

## Generate Content

In [11]:
from google.genai import types

prompt = "Draft my next blog post based on my thoughts in this audio file and these two previous blog posts I wrote."

MODEL_ID ="gemini-2.5-flash" # @param ["gemini-2.5-flash", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[prompt, blog_file, blog_file2, audio_file],
    config=types.GenerateContentConfig(
    thinking_config=types.ThinkingConfig(
      thinking_budget=0
    )
  )
)

print(response.text)

Here's a draft for your next blog post, incorporating your audio reflections and linking to your previous pieces:

---

## The Art of Thinking: Why Every "Discarded" Idea Fuels Growth

In the fast-paced world of tech and content, where new ideas are constantly emerging and old ones quickly fall by the wayside, it’s easy to feel like some of your work is simply "throwaway." I’ve certainly experienced this myself. Early in my career, I spent countless hours crafting visions, roadmaps, and detailed ideas. Often, these ambitious projects ended up being shelved or canceled – sometimes even after significant effort had already been poured into them. I remember literally throwing away large project plans, feeling a deep sense of frustration.

It still happens today, not just to me, but to my team and even entire companies. Priorities change, markets shift, and what seemed like a brilliant idea yesterday might be irrelevant tomorrow. Coming straight out of an academic environment, where every 

## Learning more

* Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) with the quickstart.